In [7]:
#Importing the libraries
import pandas as pd
#To make DataReader working otherwise it won't due to compatibility error
pd.core.common.is_list_like = pd.api.types.is_list_like 
import numpy as np

#For visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# For reading stock data from yahoo
import pandas_datareader as pdr

# For time stamps
from datetime import datetime

# For division
from __future__ import division

Questions:

    1.) What was the change in price of the stock over time?
    2.) What was the daily return of the stock on average?
    3.) What was the moving average of the various stocks?
    4.) What was the correlation between different stocks' closing prices?
    4.) What was the correlation between different stocks' daily returns?
    5.) How much value do we put at risk by investing in a particular stock?
    6.) How can we attempt to predict future stock behavior?

1.) What was the change in price of the stock over time?

In [16]:
#Company list
tech_list = ['AAPL','GOOG','MSFT','AMZN']

#Setting up the timeframe
end = datetime.now()
start = datetime(end.year - 1, end.month,end.day)

#Creating the dataframe

for stock in tech_list:   
    # Set DataFrame as the Stock Ticker
    globals()[stock] = pdr.get_data_google(start,end)

/home/alex/anaconda3/lib/python3.6/site-packages/pandas_datareader/google/daily.py:40: UnstableAPIWarning: 
The Google Finance API has not been stable since late 2017. Requests seem
to fail at random. Failure is especially common when bulk downloading.

  warnings.warn(UNSTABLE_WARNING, UnstableAPIWarning)


TypeError: object of type 'datetime.datetime' has no len()